# DFM - Tranformação das séries

> Tranformando as séries cruas em séries estacionárias com as primeiras e segundas diferenças. Ainda, os dados são agregados pela média para trimestres, de modo a deixar todas as variáveis mensais em trimestrais. 

> OBS: Outras transformações dos dados de mensal para trimestral podem ser testadas.

In [10]:
import pandas as pd
import datetime as dt
import numpy as np

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Lendo o arquivo dos dados

In [11]:
# Lendo o arquivo csv

now_data = pd.read_csv('../../data/now_data_raw.csv')
now_data.drop('Unnamed: 0', axis = 1, inplace = True)
now_data['Date'] = pd.to_datetime(now_data['Date'], format = '%Y-%m-%d')
now_data.set_index('Date', inplace = True)
now_data.rename(columns={'retail_sales':'com_sales'}, inplace=True)
now_data.shape

(249, 154)

In [12]:
# Lendo arquivo do dicionário de dados

meta_data = pd.read_csv('../../data/meta_data.csv')
transf = meta_data[['name_df', 'transf']]
transf.set_index('name_df', inplace = True)
transf.head()

,transf
name_df,
gdp,diff
ibc_br,diff
mon_gdp,diff
ind_prod,diff
ind_prod_ext,diff


# Tranformação dos dados

In [13]:
# Separando em grupos de tranformação 

non_diffs = list(transf[transf['transf'] == 'none'].index)
first_diff = list(transf[transf['transf'] == 'diff'].index)
second_diff = list(transf[transf['transf'] == 'diff_diff'].index)

# Loop para aplicar tratamentos devidos

now_data_tf = now_data.copy()

variables = now_data_tf.columns

for i in variables:
   if (i in non_diffs):
       now_data_tf[i] = now_data_tf[i]
   elif (i in first_diff):
       now_data_tf[i] = now_data_tf[i].dropna().diff()
   elif (i in second_diff):
     now_data_tf[i] = now_data_tf[i].dropna().diff().diff()

now_data_tf.head(10)

,gdp,ibc_br,mon_gdp,ind_prod,ind_prod_ext,ind_prod_transf,ind_cap_goods,ind_prod_int_goods,ind_prod_man,ind_prod_dur_goods,ind_prod_nondur_goods,ind_prod_cc_materials,ind_real_sales,vehi_prod,pass_cars_prod,truck_prod,bus_prod,steel_prod,road_flow_gross_vehi,ipca,ipca_ex,ipca_exi,ipca_exii,ipca_exiii,inpc,igpm,igpdi,incc_m,incc_di_mo,incc_di_servmat,ipatac_m,ipc_br,ipa,ipc_fipe,commdt_index_br,commdt_index_agro,commdt_index_metal,sinapi,ccb_sp,ccb_bh,ccb_rj,ccb_bsb,ccb_sal,wage_mass_ind,new_caged,caged_agro,caged_ind_ext,caged_ind_transf,caged_const,caged_com,caged_servi,hrs_wkd_ind_transf,unem_rate_pnad,total_sales,const_sales,com_sales,forn_app_sales,fuel_sales,nondur_congoods_sales,vest_sales,vehi_sales,medcare_sales,books_sales,offcomm_sales,others_sales,vehi_total_sales_fenabrave,pass_vehi_sales,light_comm_vehi_sales,truck_sales,bus_sales,vehi_total_sales_anfavea,vehi_domestic_sales,energy_comm,energy_res,energy_ind,energy_total,prod_crude_oil,prod_lgn,prod_oil_deriv,co_gas,co_glp,co_oilcomb,co_diesel,co_oilderiv_total,co_hid_alc,co_alc_total,curr_acc,idp_gdp_twelve_mon,ide_mon,net_publi_debt_states,pr_central_gov,net_debt_percent_gdp,net_debt_per_gdp_states,net_debt_per_gdp_pe,net_debt_per_gdp_cons_ps,nm_debt_cost,nm_debt_cost_int,mon_base,inter_reserv,inter_reserv_liq,mi_wkdavg,mii_balance_endp,selic_mon,interest_longterm,cdi_mon,credit_tot,credit_enter,bndes_desem_total,bndes_desem_ind_transf,bndes_desem_comserv,bndes_desem_ind_ext,exr_usd_mon_mean,household_debt,serv_conf_index_fgv,serv_curr_situ_index_fgv,serv_expec_index_fgv,ind_conf_cons_feco,ind_eco_cond_feco,ind_fut_expec_feco,emp_conf_index_fgv,emp_curr_situ_index_fgv,emp_expec_index_fgv,ind_conf_index_fgv,ind_curr_situ_index_fgv,ind_expec_index_fgv,nuci_gv,cons_conf_index_fgv,cons_curr_situ_index_fgv,cons_expec_index_fgv,expec_gdp,expec_ipca,expec_selic,expec_exr_usd,exp_fob,exp_kgood_fob,exp_comgood_fob,imp_fob,imp_kgood_fob,imp_comgood_fob,revenue_rf,brent_oil_price,ibov,nasdaq,snp_500,dji,gt_crise,gt_desemprego,gt_seguro_desemprego,gt_empregos,gt_trabalho,gt_fgts,gt_renda,gt_salario,gt_economia
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2002-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116830,110194,NaN,1810,119.87,NaN,0.52,NaN,0.74,0.44,0.44,1.07,0.36,0.19,NaN,NaN,0.70,0.14,0.79,-0.13,0.57,NaN,NaN,NaN,0.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1442,44,1486,NaN,NaN,148,NaN,NaN,NaN,NaN,NaN,NaN,8.3,NaN,-7039.63,NaN,NaN,NaN,NaN,14513,2.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.0,93.7,92.6,94.9,95.5,94.8,78.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02-01,NaN,NaN,-897.7,-2.7,-3.5,-2.6,-1.3,-2.4,-3.5,-1.8,-4.0,NaN,NaN,116000,108600,519.0,2055,113.72,1.84,0.36,0.34,0.73,0.42,0.46,0.31,0.06,0.18,-0.09,0.69,0.47,-0.09,0.14,0.14,0.26,-0.32,-0.41,NaN,0.38,0.58,3.39,2.98,1.99,-0.24,NaN,NaN,10423.0,NaN,8955.0,NaN,NaN,NaN,NaN,NaN,-2.85,NaN,NaN,-4.39,NaN,-2.59,-1.89,-7.19,NaN,NaN,NaN,NaN,-17283.0,-13188.0,-3391.0,-714.0,10.0,-9570.0,-14439.0,69.0,-68.0,488.0,371.0,1435,41,1476,9.0,16.0,126,26.0,29.0,NaN,5.0,103.6,0.01,347.1,-364.19,-2144.53,-0.53,-0.14,-0.11,-0.85,-124,0.03,-2241514.0,-261.0,-291.0,-2829283.0,878056.0,-0.28,0.0,-0.28,NaN,NaN,2559.0,768.0,1537.0,2.0,0.0417,NaN,NaN,NaN,NaN,3.62,3.64,3.61,97.1,93.8,100.5,98.9,95.6,102.5,79.2,NaN,NaN,NaN,0.05,0.07,-0.47,0.00,NaN,-294.037991,132.100389,-402.864158,-147.263236,NaN,-5177.194064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-03-01,NaN,NaN,6967.6,7.7,8.9,7.6,7.0,8.5,6.5,9.1,5.7,NaN,NaN,143887,135421,1005.0,2116,121.24,0.03,0.60,-0.40,0.52,0.51,0.54,0.62,0.09,0.11,0.53,0.06,0.35,-0.14,0.42,-0.11,0.07,-2.18,-3.82,-1.18,0.87,0.68,-5.41,0.2

# Teste final de estacionariedade

In [14]:
# Definindo a função para realizar o ADF test 

def adfuller_test(series, signif = 0.05):

# Perform Augmented Dickey-Fuller to test for Stationarity of the given series
# and print report. Null Hypothesis [H0]: Data has unit root and is non-stationary.

# series: time series in pd.Series format
# signif: significance level for P-value to reject Null Hypothesis

    x = adfuller(series, 
                 regression = 'ct',            # inserindo uma constante e um termo de tendência na regressão
                 autolag = 'BIC')              # inserindo o critério bayesiano de informação para determinaro 
                                               # número de lags em cada regressão do teste
                                               # Seguimos com o critério BIC para a inclusão automática de lags como feito em Issler (2016).

    #using dictionary saves different data types (float, int, boolean)
    output = {'test_statistic': x[0], 
              'p_value': x[1], 
              'n_lags': x[2], 
              'n_observations': x[3],
              f'reject_005': x[1] < signif }

    for key, val in x[4].items():
         output[f'Critical value {key}'] = val

    return pd.Series(output)

In [15]:
# Realizando o teste ADF

adf_tf = pd.DataFrame()
cols = now_data_tf.columns

for column in cols:
  adf_test = adfuller_test(now_data_tf[column].dropna())
  adf_tf[column] = adf_test

adf_tf_t = adf_tf.T
adf_tf_t[adf_tf_t['reject_005'] == False]

C:\Users\HENRI\AppData\Local\Temp\ipykernel_5908\2946497049.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adf_tf[column] = adf_test
C:\Users\HENRI\AppData\Local\Temp\ipykernel_5908\2946497049.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adf_tf[column] = adf_test
C:\Users\HENRI\AppData\Local\Temp\ipykernel_5908\2946497049.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

,test_statistic,p_value,n_lags,n_observations,reject_005,Critical value 1%,Critical value 5%,Critical value 10%


> Nenhuma variável apareceu no conjunto de séries que falhou em rejeitar a hipótese nula de que a série é não estacionária e que possui raíz unitária. Logo, os procedimentos para estacionarizar foram bem sucedidos.

# Agregando as séries temporais

In [16]:
# Agregando as séries pela média e substituindo NAs pela mediana

qdata_tf = (now_data_tf.groupby(pd.PeriodIndex(now_data_tf.index, freq = 'Q'), axis = 0)).mean()
cols = qdata_tf.columns

for col in cols:
    qdata_tf[col].fillna(qdata_tf[col].median(), inplace = True)

qdata_tf.head(10)
qdata_tf.shape

(83, 154)

# Salvando a base de dados nova tranformada: mensal e apenas PIB trimestral 

In [17]:
data_tf = now_data_tf.copy()
cols = data_tf.columns[1:]

for col in cols:
    data_tf[col].fillna(data_tf[col].median(), inplace = True)

data_tf

,gdp,ibc_br,mon_gdp,ind_prod,ind_prod_ext,ind_prod_transf,ind_cap_goods,ind_prod_int_goods,ind_prod_man,ind_prod_dur_goods,ind_prod_nondur_goods,ind_prod_cc_materials,ind_real_sales,vehi_prod,pass_cars_prod,truck_prod,bus_prod,steel_prod,road_flow_gross_vehi,ipca,ipca_ex,ipca_exi,ipca_exii,ipca_exiii,inpc,igpm,igpdi,incc_m,incc_di_mo,incc_di_servmat,ipatac_m,ipc_br,ipa,ipc_fipe,commdt_index_br,commdt_index_agro,commdt_index_metal,sinapi,ccb_sp,ccb_bh,ccb_rj,ccb_bsb,ccb_sal,wage_mass_ind,new_caged,caged_agro,caged_ind_ext,caged_ind_transf,caged_const,caged_com,caged_servi,hrs_wkd_ind_transf,unem_rate_pnad,total_sales,const_sales,com_sales,forn_app_sales,fuel_sales,nondur_congoods_sales,vest_sales,vehi_sales,medcare_sales,books_sales,offcomm_sales,others_sales,vehi_total_sales_fenabrave,pass_vehi_sales,light_comm_vehi_sales,truck_sales,bus_sales,vehi_total_sales_anfavea,vehi_domestic_sales,energy_comm,energy_res,energy_ind,energy_total,prod_crude_oil,prod_lgn,prod_oil_deriv,co_gas,co_glp,co_oilcomb,co_diesel,co_oilderiv_total,co_hid_alc,co_alc_total,curr_acc,idp_gdp_twelve_mon,ide_mon,net_publi_debt_states,pr_central_gov,net_debt_percent_gdp,net_debt_per_gdp_states,net_debt_per_gdp_pe,net_debt_per_gdp_cons_ps,nm_debt_cost,nm_debt_cost_int,mon_base,inter_reserv,inter_reserv_liq,mi_wkdavg,mii_balance_endp,selic_mon,interest_longterm,cdi_mon,credit_tot,credit_enter,bndes_desem_total,bndes_desem_ind_transf,bndes_desem_comserv,bndes_desem_ind_ext,exr_usd_mon_mean,household_debt,serv_conf_index_fgv,serv_curr_situ_index_fgv,serv_expec_index_fgv,ind_conf_cons_feco,ind_eco_cond_feco,ind_fut_expec_feco,emp_conf_index_fgv,emp_curr_situ_index_fgv,emp_expec_index_fgv,ind_conf_index_fgv,ind_curr_situ_index_fgv,ind_expec_index_fgv,nuci_gv,cons_conf_index_fgv,cons_curr_situ_index_fgv,cons_expec_index_fgv,expec_gdp,expec_ipca,expec_selic,expec_exr_usd,exp_fob,exp_kgood_fob,exp_comgood_fob,imp_fob,imp_kgood_fob,imp_comgood_fob,revenue_rf,brent_oil_price,ibov,nasdaq,snp_500,dji,gt_crise,gt_desemprego,gt_seguro_desemprego,gt_empregos,gt_trabalho,gt_fgts,gt_renda,gt_salario,gt_economia
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2002-01-01,NaN,-0.32,1747.5,0.0,0.1,0.25,0.4,0.15,0.4,1.35,0.15,0.75,0.4,116830,110194,161.0,1810,119.87,0.15,0.52,0.005,0.74,0.44,0.44,1.07,0.36,0.19,0.02,0.00,0.70,0.14,0.79,-0.13,0.57,0.50,0.375,0.41,0.64,1.245,1.91,1.365,0.93,1.63,2.3,-2698.0,4396.5,-71.0,20940.5,-2971.0,6294.0,-349.0,0.5,0.000000e+00,0.865,-0.045,0.815,1.14,-0.17,-0.145,1.765,0.865,0.195,-0.62,0.40,1.305,2962.5,2092.0,493.0,17.0,12.0,4243.5,2750.0,48.0,39.0,48.0,88.0,1442,44,1486,4.5,0.0,148,3.5,3.0,2.0,5.0,-119.4,0.00,8.3,1739.77,-7039.63,0.015,-0.05,0.00,-0.05,14513,2.40,611518.5,1234.5,861.0,958387.0,13073481.5,0.00,0.00,0.00,5595.0,6610.0,5991.0,1784.0,3307.0,24.0,-0.0043,0.055,-0.1,0.3,-0.3,0.275,0.345,0.065,93.0,93.7,92.6,94.9,95.5,94.8,78.6,0.45,0.1,0.5,-0.0100,0.0300,0.00,0.0000,147.265871,37.932125,46.826098,72.203004,14.515333,26.404655,2131.896145,1.230000,466.0,67.299805,22.630005,165.940430,0.0,18.0,15.0,-2.0,1.0,7.0,0.0,0.0,0.0
2002-02-01,NaN,-0.32,-897.7,-2.7,-3.5,-2.60,-1.3,-2.40,-3.5,-1.80,-4.00,0.75,0.4,116000,108600,519.0,2055,113.72,1.84,0.36,0.340,0.73,0.42,0.46,0.31,0.06,0.18,-0.09,0.69,0.47,-0.09,0.14,0.14,0.26,-0.32,-0.410,0.41,0.38,0.580,3.39,2.980,1.99,-0.24,2.3,-2698.0,10423.0,-71.0,8955.0,-2971.0,6294.0,-349.0,0.5,0.000000e+00,-2.850,-0.045,0.815,-4.39,-0.17,-2.590,-1.890,-7.190,0.195,-0.62,0.40,1.305,-17283.0,-13188.0,-3391.0,-714.0,10.0,-9570.0,-14439.0,69.0,-68.0,488.0,371.0,1435,41,1476,9.0,16.0,126,26.0,29.0,2.0,5.0,103.6,0.01,347.1,-364.19,-2144.53,-0.530,-0.14,-0.11,-0.85,-124,0.03,-2241514.0,-261.0,-291.0,-2829283.0,878056.0,-0.28,0.00,-0.28,5595.0,6610.0,2559.0,768.0,1537.0,2.0,0.0417,0.055,-0.1,0.3,-0.3,3.620,3.640,3.610,97.1,93.8,100.5,98.9,95.6,102.5,79.2,0.45,0.1,0.5,0.0500,0.0700,-0.47,0.0000,147.26

### Salvando em .csv os dados transformados

In [18]:
# now_data_tf.to_csv('now_data_tf.csv')
# now_data_tf.to_excel('now_data_tf.xlsx')

qdata_tf.to_csv('../../data/qdata_tf.csv')
qdata_tf.to_excel('../../data/qdata_tf.xlsx')

data_tf.to_csv('../../data/data_tf.csv')
data_tf.to_excel('../../data/data_tf.xlsx')